In [ ]:
# Block 1: Mount Google Drive
from google.colab import drive

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Block 2: Load the original dataset (Excel file)
import pandas as pd

# Adjust the path to where your Excel file is located in Google Drive
file_path = '/content/drive/MyDrive/Skunkworks_Theriver_raw_v2.xlsx'  # Change this to your file path
df = pd.read_excel(file_path)


In [ ]:
# Block 3: Process the DataFrame
# Convert 'Amount' to numeric (remove $ and commas)
# Assuming your date column is named 'Date'
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')  # Convert to datetime, coerce errors to NaT

df['Amount'] = df['Amount'].replace({'\$': '', ',': ''}, regex=True).astype(float)

# Filter out rows with $0 donations
df_v3 = df[df['Amount'] > 0]

# Impute missing values in the 'Appeal' column
df_v3['Primary ZIP Code'].fillna('Unknown', inplace=True)


<ipython-input-28-613f1148effc>:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_v3['Primary ZIP Code'].fillna('Unknown', inplace=True)
<ipython-input-28-613f1148effc>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_v3['Primary ZIP Code'].fillna('Unknown', inplace=True)


In [ ]:
# Block 4: Save the V3 dataset
# You can save it back to Google Drive
v3_file_path = '/content/drive/MyDrive/V3DataSet.'  # Change this to your desired save path
df_v3.to_csv(v3_file_path, index=False)

print("V3 dataset created successfully and saved to Google Drive!")


V3 dataset created successfully and saved to Google Drive!


In [ ]:
# Group by 'Campaign' to calculate total donations
total_per_campaign = df.groupby('Campaign')['Amount'].sum().reset_index()
total_per_campaign.rename(columns={'Amount': 'Total Donation'}, inplace=True)

# Merge total donations back into the original DataFrame
df = pd.merge(df, total_per_campaign, on='Campaign', how='left')


In [ ]:
# Assuming the 'Date' column is in datetime format
# Extract min and max date for each campaign
campaign_dates = df.groupby('Campaign')['Date'].agg(['min', 'max']).reset_index()
campaign_dates['Num Months'] = ((campaign_dates['max'] - campaign_dates['min']).dt.days / 30).round()

# Merge the number of months back into the original DataFrame
df = pd.merge(df, campaign_dates[['Campaign', 'Num Months']], on='Campaign', how='left')


In [ ]:
# Calculate Average Monthly Donation
df['Average Monthly Donation'] = df['Total Donation'] / df['Num Months']


In [ ]:
# Optional: Clean up the DataFrame to keep only relevant columns
df_cleaned = df[['Date', 'Amount', 'Campaign', 'Total Donation', 'Num Months', 'Average Monthly Donation']]

# Save the updated DataFrame to a new Excel file
output_file_path = '/content/drive/MyDrive/V3.1DataSet_Normization(Including Montly averages).xlsx'  # Adjust the output path
df_cleaned.to_excel(output_file_path, index=False)

print("Updated dataset with total and average monthly donations has been saved successfully!")


Updated dataset with total and average monthly donations has been saved successfully!


In [ ]:
# MORE DATA MANIPULATIOn

In [ ]:
import pandas as pd

# Load your CSV sheet (replace with your actual file path)
file_path = '/content/drive/MyDrive/V3DataSet.csv'  # Update this path if necessary

try:
    df = pd.read_csv(file_path)  # Load CSV file
    print("Original DataFrame:")
    print(df)

    # Step 1: Calculate the total amount
    total_amount = df['Amount'].sum()

    # Step 2: Normalize the Amount column and create a new column
    df['Normalized Amount'] = df['Amount'] / total_amount

    # Display the updated DataFrame
    print("\nDataFrame with Normalized Amount:")
    print(df)

    # Save the updated DataFrame back to a CSV file
    output_file_path = '/content/drive/MyDrive/V3DataSet.csv'  # Update this path if necessary
    df.to_csv(output_file_path, index=False)

    print(f"\nNormalized data saved to: {output_file_path}")

except Exception as e:
    print(f"An error occurred: {e}")


Original DataFrame:
             Date      Amount            Type                   Fund  \
0      2024-06-10  3000000.00          Pledge  Cap Camp Unrestricted   
1      2024-03-12  1500000.00  Pledge Payment  Cap Camp Unrestricted   
2      2023-08-21  1500000.00          Pledge  Cap Camp Unrestricted   
3      2023-07-27  1012500.00        Donation           Unrestricted   
4      2024-05-02   660000.00          Pledge           Unrestricted   
...           ...         ...             ...                    ...   
10946  2023-11-01        0.75        Donation           Unrestricted   
10947  2024-06-03        0.70        Donation           Unrestricted   
10948  2023-01-06        0.37        Donation           Unrestricted   
10949  2024-08-01        0.20        Donation           Unrestricted   
10950  2024-09-04        0.11        Donation           Unrestricted   

                        Campaign                    Appeal Primary ZIP Code  \
0      2023-2024 Recipe for Hope    

In [ ]:
import pandas as pd

# Load the ZIP code data from the specified path
zip_file_path = '/content/drive/MyDrive/reorganized_median_income_data.csv'
zip_data = pd.read_csv(zip_file_path)

# Load the main donation data
donation_file_path = '/content/drive/MyDrive/Skunkworks_Theriver_raw_v2.xlsx'  # Update with your actual file path
donation_data = pd.read_excel(donation_file_path, engine='openpyxl')

# Display the initial DataFrames
print("ZIP Data:")
print(zip_data.head())
print("\nDonation Data:")
print(donation_data.head())

# Convert both merge keys to string type for compatibility
donation_data['Primary ZIP Code'] = donation_data['Primary ZIP Code'].astype(str)
zip_data['Zip'] = zip_data['Zip'].astype(str)

# Merge the datasets on the ZIP code
merged_data = pd.merge(donation_data, zip_data, left_on='Primary ZIP Code', right_on='Zip', how='left')

# Display the merged DataFrame
print("\nMerged Data:")
print(merged_data.head())

# Optionally save the merged data
merged_file_path = '/content/drive/MyDrive/MergedData.csv'  # Update with your desired file path
merged_data.to_csv(merged_file_path, index=False)

print(f"\nMerged data saved to: {merged_file_path}")


ZIP Data:
     Zip  Median Household Income Population  Rank
0  53122                 119536.0      6,080   1.0
1  54169                 101339.0      2,653   2.0
2  53069                  98625.0        524   3.0
3  53217                  97690.0     29,496   4.0
4  53097                  96310.0      5,534   5.0

Donation Data:
        Date     Amount            Type                   Fund  \
0 2024-06-10  3000000.0          Pledge  Cap Camp Unrestricted   
1 2024-03-12  1500000.0  Pledge Payment  Cap Camp Unrestricted   
2 2023-08-21  1500000.0          Pledge  Cap Camp Unrestricted   
3 2023-07-27  1012500.0        Donation           Unrestricted   
4 2024-05-02   660000.0          Pledge           Unrestricted   

                    Campaign                  Appeal Primary ZIP Code  \
0  2023-2024 Recipe for Hope         Gov Application              NaN   
1  2023-2024 Recipe for Hope         Gov Application       53703-3340   
2  2023-2024 Recipe for Hope         Gov Application

In [ ]:
#v2 vs v3

In [ ]:
# Install MAST-ML and required dependencies
!pip install mastml
!pip install scikit-learn==1.1.3


  Using cached scikit_learn-1.5.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
Using cached scikit_learn-1.5.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.3 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.1.3
    Uninstalling scikit-learn-1.1.3:
      Successfully uninstalled scikit-learn-1.1.3
  Using cached scikit_learn-1.1.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
Using cached scikit_learn-1.1.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (30.5 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.5.2
    Uninstalling scikit-learn-1.5.2:
      Successfully uninstalled scikit-learn-1.5.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.19.0 requires scikit-learn>=1.2.2, but 

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
# Import required modules for MAST-ML and data handling
from google.colab import drive
from mastml.mastml import Mastml
from mastml.preprocessing import SklearnPreprocessor
from mastml.models import SklearnModel
from mastml.data_splitters import SklearnDataSplitter
from xgboost import XGBRegressor
import pandas as pd


In [ ]:
# Load your dataset
file_path = '/content/drive/MyDrive/V3DataSet_Normization.xlsx'
data = pd.read_excel(file_path)

# Drop categorical columns and focus on numeric data
data = data.select_dtypes(include=['float64', 'int64'])

# Step 6: Check for NaN and Infinite Values
print("NaN values in dataset:\n", data.isna().sum())
print("Infinite values in dataset:\n", np.isinf(data).sum())

# Handle NaN and Infinite Values
data = data.fillna(0)  # Fill NaN values with 0, or use data.dropna()
data = data[~np.isinf(data).any(axis=1)]  # Remove rows with infinite values


NaN values in dataset:
 Amount                      0
Normalized Amount           0
Account Number              0
Total Donation              0
Num Months                  0
Average Monthly Donation    0
dtype: int64
Infinite values in dataset:
 Amount                       0
Normalized Amount            0
Account Number               0
Total Donation               0
Num Months                   0
Average Monthly Donation    19
dtype: int64


In [ ]:
# Step 7: Set Up Save Path
SAVEPATH = '/content/drive/MyDrive/V3_regression_forest_data'
mastml = Mastml(savepath=SAVEPATH)
savepath = mastml.get_savepath  # Note: no parentheses

/content/drive/MyDrive/V3_regression_forest_data not empty. Renaming...


In [ ]:
# Step 8: Preprocess Data
preprocessor = SklearnPreprocessor(preprocessor='StandardScaler', as_frame=True)
X = preprocessor.evaluate(data, savepath=savepath)


In [ ]:
# Step 9: Define and Evaluate Models
model1 = SklearnModel(model='LinearRegression')
model2 = SklearnModel(model='KernelRidge', kernel='rbf')
model3 = SklearnModel(model='RandomForestRegressor')

# Define XGBoost model without SklearnModel
xgboost_model = XGBRegressor()  # Create an instance of XGBRegressor

models = [model1, model2, model3]  # Add other models to the list

In [ ]:
# Assuming 'Total Donation' is your target variable
target_variable = 'Total Donation'
X_features = data.drop(columns=[target_variable])  # Features without target

# Step 10: Split and Evaluate Models
splitter = SklearnDataSplitter(splitter='RepeatedKFold', n_repeats=1, n_splits=5)
result = splitter.evaluate(X=X_features, y=data[target_variable], models=models + [xgboost_model], preprocessor=preprocessor, savepath=savepath, verbosity=3)

Running splits: 100%|██████████| 5/5 [00:07<00:00,  1.60s/it]


Running splits: 100%|██████████| 5/5 [01:22<00:00, 16.49s/it]


Running splits: 100%|██████████| 5/5 [06:36<00:00, 79.40s/it]


Running splits: 100%|██████████| 5/5 [00:08<00:00,  1.71s/it]


In [ ]:
print(f"Results saved at: {savepath}")


Results saved at: /content/drive/MyDrive/V3_regression_forest_data_2024_10_06_20_40_19
